# Day 8

In [109]:
import os

In [113]:
input_file_path = os.path.join(".", "day08.txt")
with open(input_file_path, 'r') as reader:
    input_data = reader.read()

## Part 1
Count instances of 1, 4, 7 and 8

In [114]:
count = 0
for line in input_data.split("\n"):
    segments, reading = line.split(" | ")

    # Pressence of 1, 4, 7 and 8 digits
    count += sum(1 for x in reading.split() if len(x) in (2, 4, 3, 7))

print(f"Part 1: {count:,} appearances")

Part 1: 532 appearances


## Part 2
Deduce all readings

In [115]:
digit_lookup = {
    1: "cf",      # 2 segments
    4: "bcdf",    # 4 segments
    7: "acf",     # 3 segments
    8: "abcdefg", # 7 segments

    2: "acdeg",   # 5 segments
    3: "acdfg",   # 5 segments
    5: "abdfg",   # 5 segments

    0: "abcefg",  # 6 segments
    6: "abdefg",  # 6 segments
    9: "abcdfg",  # 6 segments
}

total = 0
for line in input_data.split("\n"):
    segments, reading = line.split(" | ")

    segment_mapping = {}
    unknown_five_segments = set()
    unknown_six_segments = set()

    # Map the segments
    for segment in segments.split(" "):
        n = len(segment)

        # Trivial 1:1 segments
        if n == 2:
            segment_mapping[1] = set(segment)
        elif n == 4:
            segment_mapping[4] = set(segment)
        elif n == 3:
            segment_mapping[7] = set(segment)
        elif n == 7:
            segment_mapping[8] = set(segment)

        # Possible mappings to 2, 3, 5
        elif n == 5:
            unknown_five_segments.add(segment)

        # Possible mappings to 0, 6, 9
        elif n == 6:
            unknown_six_segments.add(segment)

        else:
            raise ValueError(f"Unknown segment detected in {segments}")

    # Deduce number: 3
    tmp_three = [x for x in unknown_five_segments if not segment_mapping[7] - set(x)][0]
    segment_mapping[3] = set(tmp_three)
    unknown_five_segments.remove(tmp_three)

    # Deduce segments: a, b, d, e, g
    segment_a = segment_mapping[7] - segment_mapping[1]
    segment_g = segment_mapping[3] - segment_mapping[4] - segment_a
    segment_d = segment_mapping[3] - segment_mapping[7] - segment_g
    segment_b = segment_mapping[4] - segment_mapping[7] - segment_d
    segment_e = segment_mapping[8] - segment_mapping[3] - segment_b

    # Deduce number: 0
    tmp_zero = [x for x in unknown_six_segments if segment_mapping[8] - segment_d == set(x)][0]
    segment_mapping[0] = set(tmp_zero)
    unknown_six_segments.remove(tmp_zero)

    # Deduce segments c, f
    # Deduce numbers: 6, 9
    tmp_segments = segment_a | segment_d | segment_g | segment_b | segment_e
    tmp_six, tmp_nine = unknown_six_segments
    tmp_six, tmp_nine = set(tmp_six), set(tmp_nine)
    if len(tmp_six - tmp_segments) < 2:
        segment_f = tmp_six - tmp_segments
        segment_mapping[6] = tmp_six
        segment_mapping[9] = tmp_nine
    else:
        segment_f = tmp_nine - tmp_segments
        segment_mapping[6] = tmp_nine
        segment_mapping[9] = tmp_six
    segment_c = segment_mapping[9] - segment_mapping[6]

    # Infer numbers: 2, 5
    segment_mapping[2] = segment_a | segment_c | segment_d | segment_e | segment_g
    segment_mapping[5] = segment_a | segment_b | segment_d | segment_f | segment_g

    # Build reverse lookup map
    digit_lookup = {
        "".join(sorted(v)): k
        for k, v in segment_mapping.items()
    }

    # Parse reading
    digits = [digit_lookup["".join(sorted(x))] for x in reading.split(" ")]
    total += sum(x * y for x, y in zip(digits, [1_000, 100, 10, 1]))

print(f"Part 2: {total:,} total")

Part 2: 1,011,284 total
